# Google Colab用 ComfyUI 起動ノートブック（ワンセル・最終安全版）

**特徴：**
- **ワンセル（1つのセル）**で、すべてのセットアップが完了します。
- Googleドライブと連携し、モデルや生成画像を永続的に保存できます。
- エラーが起きにくい、安定した構成になっています。

**使い方：**
1. 最初に表示されるポップアップで、**Googleドライブへのアクセスを許可**してください。
2. 下のコードセル（1つだけです）の左側にある、**実行ボタン（▷）**をクリックしてください。
3. 初回は10〜15分ほどかかります。じっくりお待ちください。
4. ログの最後に表示される、**`...loca.lt`** というURLをクリックして、ComfyUIにアクセスしてください。

In [ ]:
# ================================================================
#  ComfyUI for Google Colab – Fixed Version
# ================================================================
from google.colab import drive
import os, subprocess, sys, time, threading

# 1) Googleドライブをマウント ---------------------------------------------
print("■ Step-1: Googleドライブをマウントします...")
drive.mount('/content/drive')

# 2) ComfyUI本体とManagerをGoogleドライブにインストール ----------------
print("\n■ Step-2: ComfyUI本体とManagerをインストールします...")
COMFY_DIR = '/content/drive/MyDrive/ComfyUI'
if not os.path.isdir(COMFY_DIR):
    get_ipython().system('git clone --depth 1 https://github.com/comfyanonymous/ComfyUI.git {COMFY_DIR}')
else:
    get_ipython().system('cd {COMFY_DIR} && git pull')

MGR_DIR = f"{COMFY_DIR}/custom_nodes/ComfyUI-Manager"
if not os.path.isdir(MGR_DIR):
    get_ipython().system('git clone --depth 1 https://github.com/ltdrdata/ComfyUI-Manager.git {MGR_DIR}')

# 3) Python依存ライブラリのインストール --------------------------------
print("\n■ Step-3: 必要なライブラリをインストールします...")
get_ipython().system('pip -q install --upgrade pip wheel setuptools')
get_ipython().system('pip -q install torch==2.1.2 torchvision==0.16.1 torchaudio==2.1.2 --index-url https://download.pytorch.org/whl/cu118')
get_ipython().system('pip -q install xformers==0.0.23.post1 triton==2.1.0 numpy Pillow toml')
get_ipython().system('pip -q install -r {COMFY_DIR}/requirements.txt')

# 4) Base Model (SD-1.5) をGoogleドライブにダウンロード -----------------
print("\n■ Step-4: 基本モデル（SD-1.5）をダウンロードします...")
MODEL_DST = f"{COMFY_DIR}/models/checkpoints"
os.makedirs(MODEL_DST, exist_ok=True)
model_file = f"{MODEL_DST}/v1-5-pruned-emaonly.safetensors"
if not os.path.exists(model_file):
    get_ipython().system('wget -q --show-progress -O {model_file} https://huggingface.co/runwayml/stable-diffusion-v1-5/resolve/main/v1-5-pruned-emaonly.safetensors')

# 5) 起動とトンネル作成 -----------------------------------------------------
print("\n✅  準備完了！ これから ComfyUI を起動します。")
print("   1) このセルの実行が終わり、URLが表示されるまでお待ちください。")
print("   2) 表示された `...loca.lt` というURLをクリックしてください。")

get_ipython().system('npm install -g localtunnel')

def run_comfyui():
    os.chdir(COMFY_DIR)
    os.system("python main.py --listen 127.0.0.1 --port 8123")

thread = threading.Thread(target=run_comfyui, daemon=True)
thread.start()

print("\n⏳ ComfyUIの起動を待っています（15秒）...")
time.sleep(15)

print("\n🌐 トンネルを作成中...")
get_ipython().system('lt --port 8123')
